# LMSYS - Chatbot Arena Human Preference Predictions



**Due to the size of the train data, and I only using 0.5% of the train data!**

WIP: Compute embeddings using TPU in a differente notebook to use the full train data and then load the embeddings here!

## Install and load libraries

In [ ]:
!pip install textstat SweetViz

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import sweetviz as sv

import re
import string
import nltk
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.sentiment.vader import SentimentIntensityAnalyzer

import textstat
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split

from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, TFBertModel

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import accuracy_score, \
                            log_loss, \
                            confusion_matrix, \
                            classification_report

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

## Loading data

In [ ]:
train_data = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
test_data = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

## Exploratory Data Analysis (EDA)

In [ ]:
train_analysis = sv.analyze(train_data)

In [ ]:
train_analysis.show_html('train_analysis.html')

In [ ]:
train_data.head()

In [ ]:
print("Training Data -", train_data.shape)
print("Test Data -", test_data.shape)

In [ ]:
train_data.describe(include=['O'])

In [ ]:
print(train_data.info())

In [ ]:
train_data.drop("id", axis=1).duplicated().sum()

There exist 14 duplicated rows forming 7 groups, I will just keep one row per group.

In [ ]:
train_data = train_data.drop_duplicates(keep="first", ignore_index=True)

Checking the quality of the train data with respect the `id` column.

In [ ]:
train_data.nunique()

In [ ]:
assert train_data["id"].nunique() == len(train_data)

In [ ]:
train_data.isna().sum()

### Distribution

In [ ]:
model_df = pd.concat([train_data.model_a, train_data.model_b])
counts = model_df.value_counts().reset_index()
counts.columns = ['LLM', 'Count']

# Create a bar plot with custom styling using Plotly
fig = px.bar(counts, x='LLM', y='Count',
             title='Distribution of LLMs',
             color='Count')

fig.update_layout(xaxis_tickangle=-45)  # Rotate x-axis labels for better readability

fig.show()

In [ ]:
counts_a = train_data['winner_model_a'].value_counts().reset_index()
counts_b = train_data['winner_model_b'].value_counts().reset_index()
counts_tie = train_data['winner_tie'].value_counts().reset_index()

# Renaming columns for convinience
counts_a.columns = ['Winner', 'Count']
counts_b.columns = ['Winner', 'Count']
counts_tie.columns = ['Winner', 'Count']

# Adding column to identify the model
counts_a['Model'] = 'Model A'
counts_b['Model'] = 'Model B'
counts_tie['Model'] = 'Tie'

counts = pd.concat([counts_a, counts_b, counts_tie])

fig = px.bar(counts, x='Model', y='Count', 
             color='Model',
             title='Winner Distribution for Train Data',
             labels={'Model': 'Model', 'Count': 'Win Count', 'Winner': 'Winner'})

fig.update_layout(xaxis_title="Model", yaxis_title="Win Count")

fig.show()

Conclusions:

* There are 57477 training rows and 3 test rows.
    * Note: Test data will be replaced with the full test set (~25k rows, 70% for private LB) during scoring phase.
* The column `id` has no duplicated values.
* Model identities aren't revealed in the test set.
* Strings in columns prompt, `response_a`, and `response_a` are wrapped in a list. 
    * The reason is that each chat can contains more than one prompt/response pairs.
* After dropping `id` column, there exist 14 duplicated rows forming 7 groups, we just keep one row per group and shape of the training DataFrame becomes (57470, 8).

## Data preparation and Feature Engineering

* Cleaning data: clean text, such as removing special characters, normalizing to lowercase, removing stopwords and tokenizing.
* Tokenize Inputs: using the TensorFlow/Kerar tokenizer by training on training data and fitting on both training and test data.
* Padding sequences to `max_len`.
* Create BERT embeddings.
* Compute Similarity Features using BERT between the prompt and responses for each model. 
* Compute word count, character count, and lexical diversity for each response.
* Tokenize the text inputs for the BERT model.

In [ ]:
train_data = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv').sample(frac=0.001)

### Cleaning data

Clean text, such as removing special characters, normalizing to lowercase and removing stopwords.

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [ ]:
# Cleaning texts
train_data['prompt_clean'] = train_data['prompt'].apply(clean_text)
train_data['response_a_clean'] = train_data['response_a'].apply(clean_text)
train_data['response_b_clean'] = train_data['response_b'].apply(clean_text)

### Tokenize Inputs

Using the TensorFlow/Kerar tokenizer on both training and test data. Padding sequences to `max_len`.

In [ ]:
max_len = 512

In [ ]:
tokenizer = Tokenizer(num_words=20000)

In [ ]:
tokenizer.fit_on_texts(pd.concat([train_data['prompt_clean'], train_data['response_a_clean'], train_data['response_b_clean']]))

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_data['prompt_clean'])
response_a_sequences = tokenizer.texts_to_sequences(train_data['response_a_clean'])
response_b_sequences = tokenizer.texts_to_sequences(train_data['response_b_clean'])

In [ ]:
# Padding
train_sequences = pad_sequences(train_sequences, maxlen=max_len, padding='post')
response_a_sequences = pad_sequences(response_a_sequences, maxlen=max_len, padding='post')
response_b_sequences = pad_sequences(response_b_sequences, maxlen=max_len, padding='post')

### Sentiment Analysis

Sentiment analysis using `vaderSentiment`. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
def sentiment_analysis(text):
    return analyzer.polarity_scores(text)['compound']

In [ ]:
train_data['sentiment_prompt'] = train_data['prompt_clean'].apply(sentiment_analysis)
train_data['sentiment_response_a'] = train_data['response_a_clean'].apply(sentiment_analysis)
train_data['sentiment_response_b'] = train_data['response_b_clean'].apply(sentiment_analysis)

### Text Features

Calculate text features such as word count, character count, 
lexical diversity, syllable count, sentence count and 
calculating the Flesch Reading Ease score (quantitative 
measurement of how readable a piece of text is) for each response. 

I am using `textstat` library that analyze text statistics.

In [ ]:
def word_count(text):
    return len(text.split())

def char_count(text):
    return len(text)

def lexical_diversity(text):
    words = text.split()
    return len(set(words)) / len(words) if words else 0

def syllable_count(text):
    return textstat.syllable_count(text)

def sentence_count(text):
    return textstat.sentence_count(text)

def flesch_reading_ease(text):
    return textstat.flesch_reading_ease(text)

In [ ]:
train_data['word_count_prompt'] = train_data['prompt_clean'].apply(word_count)
train_data['word_count_response_a'] = train_data['response_a_clean'].apply(word_count)
train_data['word_count_response_b'] = train_data['response_b_clean'].apply(word_count)
train_data['char_count_prompt'] = train_data['prompt_clean'].apply(char_count)
train_data['char_count_response_a'] = train_data['response_a_clean'].apply(char_count)
train_data['char_count_response_b'] = train_data['response_b_clean'].apply(char_count)
train_data['lexical_diversity_prompt'] = train_data['prompt_clean'].apply(lexical_diversity)
train_data['lexical_diversity_response_a'] = train_data['response_a_clean'].apply(lexical_diversity)
train_data['lexical_diversity_response_b'] = train_data['response_b_clean'].apply(lexical_diversity)
train_data['syllable_count_prompt'] = train_data['prompt_clean'].apply(syllable_count)
train_data['syllable_count_response_a'] = train_data['response_a_clean'].apply(syllable_count)
train_data['syllable_count_response_b'] = train_data['response_b_clean'].apply(syllable_count)
train_data['sentence_count_prompt'] = train_data['prompt_clean'].apply(sentence_count)
train_data['sentence_count_response_a'] = train_data['response_a_clean'].apply(sentence_count)
train_data['sentence_count_response_b'] = train_data['response_b_clean'].apply(sentence_count)
train_data['flesch_reading_ease_prompt'] = train_data['prompt_clean'].apply(flesch_reading_ease)
train_data['flesch_reading_ease_response_a'] = train_data['response_a_clean'].apply(flesch_reading_ease)
train_data['flesch_reading_ease_response_b'] = train_data['response_b_clean'].apply(flesch_reading_ease)

### Create BERT embeddings

Compute BERT embeddings for prompt and response for both train and test data.
Also compute Cosine Similarity features using BERT between the prompt and responses for each model. 

I will use `tf.data.Dataset` to create an efficient pipeline, and process the features in batches using GPU. Also I am going to save the intermediate embeddings using `joblib` library

In [ ]:
# Load BERT
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = TFBertModel.from_pretrained(bert_model_name)

In [ ]:
@tf.function
def get_bert_embeddings(texts):
    inputs = bert_tokenizer(texts, 
                       return_tensors='tf', 
                       padding=True, 
                       truncation=True, 
                       max_length=512)
    outputs = bert_model(inputs)
    return outputs.last_hidden_state[:, 0, :]

In [ ]:
def process_column(column_data):
    column_data = column_data.dropna().tolist()
    column_data = [str(text) for text in column_data]  
    dataset = tf.data.Dataset.from_tensor_slices(column_data)
    dataset = dataset.batch(8)  
    
    embeddings = []
    for batch in dataset:
        batch_list = [str(text) for text in batch.numpy().tolist()]  
        batch_embeddings = get_bert_embeddings(batch_list)
        embeddings.append(batch_embeddings)
    
    return np.concatenate(embeddings, axis=0)

In [ ]:
def add_embeddings_to_dataframe(df, column_names):
    for column in column_names:
        print(f"Processing column: {column}")
        embeddings = process_column(df[column])
        df[f'{column}_embedding'] = list(embeddings)
    return df

In [ ]:
columns_to_embed = ['prompt_clean', 'response_a_clean', 'response_b_clean']

In [ ]:
train_data = add_embeddings_to_dataframe(train_data, columns_to_embed)

In [ ]:
train_data['similarity_prompt_response_a'] = train_data.apply(
    lambda x: cosine_similarity(np.array(x['prompt_clean_embedding']).reshape(1, -1),
                                np.array(x['response_a_clean_embedding']).reshape(1, -1))[0][0], axis=1)

train_data['similarity_prompt_response_b'] = train_data.apply(
    lambda x: cosine_similarity(np.array(x['prompt_clean_embedding']).reshape(1, -1),
                                np.array(x['response_b_clean_embedding']).reshape(1, -1))[0][0], axis=1)

### Prepare Data

In [ ]:
X = train_data[['word_count_prompt', 'word_count_response_a', 'word_count_response_b',
                'char_count_prompt', 'char_count_response_a', 'char_count_response_b',
                'lexical_diversity_prompt', 'lexical_diversity_response_a', 'lexical_diversity_response_b',
                'syllable_count_prompt', 'syllable_count_response_a', 'syllable_count_response_b',
                'sentence_count_prompt', 'sentence_count_response_a', 'sentence_count_response_b',
                'flesch_reading_ease_prompt', 'flesch_reading_ease_response_a', 'flesch_reading_ease_response_b',
                'similarity_prompt_response_a', 'similarity_prompt_response_b', 
                'sentiment_prompt', 'sentiment_response_a', 'sentiment_response_b']]


In [ ]:
# Definindo a coluna alvo
train_data['winner'] = train_data.apply(lambda x: 0 if x['winner_model_a'] == 1 else (1 if x['winner_model_b'] == 1 else 2), axis=1)

In [ ]:
y = train_data['winner']

### Splitting training and validation data

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, 
                                                  random_state=42)

## Defining Models

* Random Forest
* Logistic Regression
* Support Vector Machin
* Gradient Boosting
* Neural Network

In [ ]:
models = {
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(probability=True),
    'Gradient Boosting': GradientBoostingClassifier()
}

In [ ]:
# Creating Neural Network
def create_nn_model(input_shape):
    model = Sequential()
    model.add(Dense(128, input_shape=(input_shape,), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
nn_model = create_nn_model(X_train.shape[1])
models['Neural Network'] = nn_model

Training models and evaluating:

In [ ]:
results = {}

for name, model in models.items():
    print(f"Treinando e avaliando {name}...")
    
    if name == 'Neural Network':
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=2)
        y_pred = np.argmax(model.predict(X_val), axis=1)
        y_pred_proba = model.predict(X_val)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        y_pred_proba = model.predict_proba(X_val)

    accuracy = accuracy_score(y_val, y_pred)
    logloss = log_loss(y_val, y_pred_proba)

    results[name] = {
        'Acurácia': accuracy,
        'Log Loss': logloss,
        'Relatório de Classificação': classification_report(y_val, y_pred),
        'Matriz de Confusão': confusion_matrix(y_val, y_pred)
    }

    print(f"Acurácia de {name}: {accuracy}")
    print(f"Log Loss de {name}: {logloss}")
    print(f"Relatório de Classificação de {name}:\n{classification_report(y_val, y_pred)}")
    print(f"Matriz de Confusão de {name}:\n{confusion_matrix(y_val, y_pred)}\n")

Selecting the best model with respect the Log Loss

In [ ]:
best_model_name = max(results, key=lambda name: results[name]['Log Loss'])
best_model = models[best_model_name]

## Prediction and Submission

Preparing test data, predicting on test data and creating submission data.

In [ ]:
# Cleaning text from test data
test_data['prompt_clean'] = test_data['prompt'].apply(clean_text)
test_data['response_a_clean'] = test_data['response_a'].apply(clean_text)
test_data['response_b_clean'] = test_data['response_b'].apply(clean_text)

In [ ]:
# Tokenize text from test data
test_sequences = tokenizer.texts_to_sequences(test_data['prompt_clean'])
response_a_test_sequences = tokenizer.texts_to_sequences(test_data['response_a_clean'])
response_b_test_sequences = tokenizer.texts_to_sequences(test_data['response_b_clean'])

In [ ]:
# Padding sequences from test data
test_sequences = pad_sequences(test_sequences, maxlen=max_len, padding='post')
response_a_test_sequences = pad_sequences(response_a_test_sequences, maxlen=max_len, padding='post')
response_b_test_sequences = pad_sequences(response_b_test_sequences, maxlen=max_len, padding='post')

In [ ]:
# Sentiment analysis for test data
test_data['sentiment_prompt'] = test_data['prompt_clean'].apply(sentiment_analysis)
test_data['sentiment_response_a'] = test_data['response_a_clean'].apply(sentiment_analysis)
test_data['sentiment_response_b'] = test_data['response_b_clean'].apply(sentiment_analysis)

In [ ]:
# Creating features of text structure from test data
test_data['word_count_prompt'] = test_data['prompt_clean'].apply(word_count)
test_data['word_count_response_a'] = test_data['response_a_clean'].apply(word_count)
test_data['word_count_response_b'] = test_data['response_b_clean'].apply(word_count)
test_data['char_count_prompt'] = test_data['prompt_clean'].apply(char_count)
test_data['char_count_response_a'] = test_data['response_a_clean'].apply(char_count)
test_data['char_count_response_b'] = test_data['response_b_clean'].apply(char_count)
test_data['lexical_diversity_prompt'] = test_data['prompt_clean'].apply(lexical_diversity)
test_data['lexical_diversity_response_a'] = test_data['response_a_clean'].apply(lexical_diversity)
test_data['lexical_diversity_response_b'] = test_data['response_b_clean'].apply(lexical_diversity)
test_data['syllable_count_prompt'] = test_data['prompt_clean'].apply(syllable_count)
test_data['syllable_count_response_a'] = test_data['response_a_clean'].apply(syllable_count)
test_data['syllable_count_response_b'] = test_data['response_b_clean'].apply(syllable_count)
test_data['sentence_count_prompt'] = test_data['prompt_clean'].apply(sentence_count)
test_data['sentence_count_response_a'] = test_data['response_a_clean'].apply(sentence_count)
test_data['sentence_count_response_b'] = test_data['response_b_clean'].apply(sentence_count)
test_data['flesch_reading_ease_prompt'] = test_data['prompt_clean'].apply(flesch_reading_ease)
test_data['flesch_reading_ease_response_a'] = test_data['response_a_clean'].apply(flesch_reading_ease)
test_data['flesch_reading_ease_response_b'] = test_data['response_b_clean'].apply(flesch_reading_ease)

In [ ]:
# Embedding from test data
test_data = add_embeddings_to_dataframe(test_data, columns_to_embed)

In [ ]:
# Cosine similarity from test data
test_data['similarity_prompt_response_a'] = test_data.apply(
    lambda x: cosine_similarity(np.array(x['prompt_clean_embedding']).reshape(1, -1),
                                np.array(x['response_a_clean_embedding']).reshape(1, -1))[0][0], axis=1)

test_data['similarity_prompt_response_b'] = test_data.apply(
    lambda x: cosine_similarity(np.array(x['prompt_clean_embedding']).reshape(1, -1),
                                np.array(x['response_b_clean_embedding']).reshape(1, -1))[0][0], axis=1)

In [ ]:
X_test = test_data[['word_count_prompt', 'word_count_response_a', 'word_count_response_b',
                    'char_count_prompt', 'char_count_response_a', 'char_count_response_b',
                    'lexical_diversity_prompt', 'lexical_diversity_response_a', 'lexical_diversity_response_b',
                    'syllable_count_prompt', 'syllable_count_response_a', 'syllable_count_response_b',
                    'sentence_count_prompt', 'sentence_count_response_a', 'sentence_count_response_b',
                    'flesch_reading_ease_prompt', 'flesch_reading_ease_response_a', 'flesch_reading_ease_response_b',
                    'similarity_prompt_response_a', 'similarity_prompt_response_b', 
                    'sentiment_prompt', 'sentiment_response_a', 'sentiment_response_b']]

In [ ]:
test_pred_proba = best_model.predict(X_test)

In [ ]:
submission = pd.DataFrame(test_data['id'])
submission['winner_model_a'] = test_pred_proba[:, 0]
submission['winner_model_b'] = test_pred_proba[:, 1]
submission['winner_tie'] = test_pred_proba[:, 2]

submission.to_csv('submission.csv', index=False)

In [ ]:
submission